In [3]:
import speech_recognition as sr
from pathlib import Path
from langchain_community.document_loaders import TextLoader

In [4]:
proj_dir = Path.cwd()
models_dir = proj_dir / 'models'
data_dir = proj_dir / 'data'

In [6]:
# obtain audio from the microphone
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Say something!")
    audio = r.listen(source)

Say something!


In [7]:
audio

In [8]:
# recognize speech using Google Speech Recognition
try:
    # for testing purposes, we're just using the default API key
    # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
    # instead of `r.recognize_google(audio)`
    print("Google Speech Recognition thinks you said " + r.recognize_google(audio))
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Google Speech Recognition service; {0}".format(e))

Google Speech Recognition thinks you said in the Pantry I have apples oranges cucumber and tomatoes


In [9]:
text = r.recognize_google(audio)
with open(data_dir / 'input.txt', 'w') as file:
    file.write(text)

In [10]:
from llama_index.core import Settings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core.indices.prompt_helper import PromptHelper
from llama_index.core.node_parser import SentenceSplitter
from langchain.embeddings import HuggingFaceEmbeddings

# An embedding model used to structure text into representations
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

# PromptHelper can help deal with LLM context window and token limitations
prompt_helper = PromptHelper(context_window=2048)

# SentenceSplitter used to split our data into multiple chunks
# Only a number of relevant chunks will be retrieved and fed into LLMs
node_parser = SentenceSplitter(chunk_size=300, chunk_overlap=20)

In [11]:
from langchain.llms import GPT4All
llm = GPT4All(model=str(Path(models_dir / 'mistral-7b-openorca.gguf2.Q4_0.gguf')))

In [12]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.prompt_helper = prompt_helper
Settings.node_parser = node_parser

In [13]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

# Load data.txt into a document
document = SimpleDirectoryReader(input_files=[str(Path(data_dir / 'input.txt'))]).load_data()

# Process data (chunking, embedding, indexing) and store them
index = VectorStoreIndex.from_documents(document)

# Save the index to save time next time.
index.storage_context.persist(persist_dir=str(Path(proj_dir / 'llama-cache')))

In [14]:
query_engine = index.as_query_engine()

In [15]:
response = query_engine.query('What food is in the pantry?')
print(response)

/Users/jsh/.local/share/virtualenvs/foodmic-QmZOIYsO-python/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 The foods in the pantry are apples, oranges, cucumber, and tomatoes.
